# Requirements

In [2]:
%pip install -qU langchain-openai
%pip install -U langchain-community langgraph langgraph-checkpoint-sqlite

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 58.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Use LangChain to make an openAI api call
Just a basic API call. No agent involved

In [5]:
import getpass
import os
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI\


# Environment variable setup
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([HumanMessage(content="hi!")])
print(response.content)

Hello! How can I assist you today?


# LangChain with agent

In [11]:
import getpass
import os
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.tools import Tool


# Environment variable setup
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# Tool setup
execute_task_prompt = PromptTemplate(
    template="""Given the following overall question `{input}`.

    Perform the task by understanding the problem, extracting variables, and being smart
    and efficient. Write a detailed response that address the task.
    When confronted with choices, make a decision yourself with reasoning.
    """,
    input_variables=["input"],
)

llm = ChatOpenAI(streaming=True)
llm_chain = LLMChain(llm=llm, prompt=execute_task_prompt)
reasoning = Tool(
        name="Reason",
        func=llm_chain.run,
        description="Reason about task via existing information or understanding. Make decisions / selections from options.",
    )

# Agent setup
memory = MemorySaver()
model = ChatOpenAI(model="gpt-4o-mini")
tools = [reasoning]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Given these 3 options, which one should I choose? 1. Move to New York 2. Move to Europe 3. Buy a house")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How's life in San Francisco?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 105, 'total_tokens': 122}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9ceaa7fb-cae8-44e9-b74a-262eb5f49f68-0', usage_metadata={'input_tokens': 105, 'output_tokens': 17, 'total_tokens': 122})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1Eu9JJZNAmunLFEhPc6a1umV', 'function': {'arguments': '{"args":[1,2,3],"config":{"tags":["relocation","real estate","lifestyle"],"callbacks":[]}}', 'name': 'Reason'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 160, 'total_tokens': 195}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_rea

KeyboardInterrupt: 